© 版权所有 Wenting Tu

In [1]:
import warnings 
warnings.filterwarnings("ignore") 

# <b><span style='color:#F1C40F'>|</span> 基于近邻算法的协同过滤 - UCF</b>
本节内容修改于:

 📌 [User-basedCollaborativeFiltering](https://github.com/nzhinusoftcm/review-on-collaborative-filtering/blob/master/2.User-basedCollaborativeFiltering.ipynb)

### 导入需要的包

In [2]:
import pandas as pd
import numpy as np
import os
import sys

from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import NearestNeighbors

from scipy.sparse import csr_matrix


### 导入数据集

In [3]:
ratings  = pd.read_csv(
            './recsys_data/recsys-100k-ratings.data',
            sep='\t',
            names=["userid", "itemid", "rating", "timestamp"],
        )
ratings = ratings.sort_values(by=['userid', 'itemid']).reset_index(drop=True)
ratings = ratings.drop(columns=['timestamp'])
ratings


,userid,itemid,rating
0,1,1,5
1,1,2,3
2,1,3,4
3,1,4,3
4,1,5,3
...,...,...,...
99995,943,1067,2
99996,943,1074,4
99997,943,1188,3
99998,943,1228,3


In [4]:
movies_columns = [
            'itemid', 'title', 'release date', 'video release date', 
            'IMDb URL ', 'unknown', 'Action', 'Adventure', 'Animation',
            "Children's", 'Comedy' , 'Crime' , 'Documentary' , 'Drama' , 'Fantasy' ,
            'Film-Noir', 'Horror', 'Musical' , 'Mystery' , 'Romance' , 'Sci-Fi' ,
            'Thriller' , 'War' , 'Western' ,
        ]

movies = pd.read_csv(
        './recsys_data/recsys-100k-items.data',
        sep='|',
        names=movies_columns,
        encoding='latin-1',
    )

# drop non necessary columns. From the third to the last column
todrop = list(range(2, len(movies.columns)))
movies = movies.drop(movies.columns[todrop], axis=1)
movies

,itemid,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)
...,...,...
1677,1678,Mat' i syn (1997)
1678,1679,B. Monkey (1998)
1679,1680,Sliding Doors (1998)
1680,1681,You So Crazy (1994)


### 对userids和itemids做顺序编码

In [5]:
def ids_encoder(ratings):
    users = sorted(ratings['userid'].unique())
    items = sorted(ratings['itemid'].unique())

    # create users and items encoders
    uencoder = LabelEncoder()
    iencoder = LabelEncoder()

    # fit users and items ids to the corresponding encoder
    uencoder.fit(users)
    iencoder.fit(items)

    # encode userids and itemids
    ratings.userid = uencoder.transform(ratings.userid.tolist())
    ratings.itemid = iencoder.transform(ratings.itemid.tolist())

    return ratings, uencoder, iencoder

# prepare data
ratings, uencoder, iencoder = ids_encoder(ratings)

### 对用户级别的评分做规范化

In [6]:
def normalize():
    # compute mean rating for each user
    mean = ratings.groupby(by='userid', as_index=False)['rating'].mean()
    norm_ratings = pd.merge(ratings, mean, suffixes=('','_mean'), on='userid')
    
    # normalize each rating by substracting the mean rating of the corresponding user
    norm_ratings['norm_rating'] = norm_ratings['rating'] - norm_ratings['rating_mean']
    return mean.to_numpy()[:, 1], norm_ratings
mean, ratings = normalize()
ratings

,userid,itemid,rating,rating_mean,norm_rating
0,0,0,5,3.610294,1.389706
1,0,1,3,3.610294,-0.610294
2,0,2,4,3.610294,0.389706
3,0,3,3,3.610294,-0.610294
4,0,4,3,3.610294,-0.610294
...,...,...,...,...,...
99995,942,1066,2,3.410714,-1.410714
99996,942,1073,4,3.410714,0.589286
99997,942,1187,3,3.410714,-0.410714
99998,942,1227,3,3.410714,-0.410714


### 构造计算用户相似度的评分矩阵R

In [7]:
def get_csr_rating_matrix(ratings):   
    '''
    我们用norm_rating来做评分矩阵，后续如果使用cosine相似度，就相当于使用pearson相关系数
    '''
    return csr_matrix(pd.crosstab(ratings.userid, ratings.itemid, ratings.norm_rating, aggfunc=sum).fillna(0).values)

rating_matrix = get_csr_rating_matrix(ratings)

In [8]:
rating_matrix

<943x1682 sparse matrix of type '<class 'numpy.float64'>'
	with 99650 stored elements in Compressed Sparse Row format>

### 利用评分矩阵+cosine相似度作学习近邻模型，得到每个用户的相似用户集和对应的相似度

In [9]:
def create_model(rating_matrix, k=20, metric="cosine"):
    """
    - create the nearest neighbors model with the corresponding similarity metric
    - fit the model
    """
    model = NearestNeighbors(metric=metric, n_neighbors=k+1, algorithm='brute')
    model.fit(rating_matrix)    
    return model

In [10]:
def nearest_neighbors(rating_matrix, model):
    """    
    :param rating_matrix : rating matrix of shape (nb_users, nb_items)
    :param model : nearest neighbors model    
    :return
        - similarities : distances of the neighbors from the referenced user
        - neighbors : neighbors of the referenced user in decreasing order of similarities
    """    
    similarities, neighbors = model.kneighbors(rating_matrix)        
    return similarities[:, 1:], neighbors[:, 1:]

In [11]:
model = create_model(rating_matrix, metric='cosine') # we can also use the 'euclidian' distance
similarities, neighbors = nearest_neighbors(rating_matrix, model)

In [12]:
similarities

array([[0.79520838, 0.79767934, 0.8034083 , ..., 0.82645814, 0.82808918,
        0.8288477 ],
       [0.60625011, 0.71469001, 0.71953175, ..., 0.80376944, 0.80718126,
        0.80747815],
       [0.76843901, 0.76953166, 0.79427808, ..., 0.8603097 , 0.86617232,
        0.86766448],
       ...,
       [0.72040905, 0.74140621, 0.74514172, ..., 0.83147653, 0.83441328,
        0.83647857],
       [0.83263536, 0.83661343, 0.83855341, ..., 0.87313794, 0.8742007 ,
        0.87500949],
       [0.77431478, 0.79267748, 0.79864397, ..., 0.83911277, 0.84025399,
        0.84393275]])

In [13]:
similarities.shape

(943, 20)

In [14]:
neighbors

array([[772, 867, 591, ..., 549,  12, 660],
       [650,  33, 309, ..., 623, 128, 419],
       [807, 686, 655, ..., 546, 709, 840],
       ...,
       [340,   3, 510, ..., 851, 819, 733],
       [682, 124, 663, ..., 332, 553, 328],
       [932, 193, 346, ..., 245, 822, 215]], dtype=int64)

### 定义找到推荐候选商品的函数

在这里我们定义以下商品为候选推荐商品：

- 不属于 ```active_items``` (目标用户已经购买过的商品)
- 属于 ```Gu_items``` (相似用户已经**频繁**购买过的商品)
- 属于相似用户购买频率top的商品

In [15]:
def find_candidate_items(userid, candidate_number=30):
    """
    Find candidate items for an active user
    
    :param userid : active user
    :param neighbors : users similar to the active user        
    :return candidates : top 30 of candidate items
    """
    user_neighbors = neighbors[userid]
    activities = ratings.loc[ratings.userid.isin(user_neighbors)]
    
    # sort items in decreasing order of frequency
    frequency = activities.groupby('itemid')['rating'].count().reset_index(name='count').sort_values(['count'],ascending=False)
    Gu_items = frequency.itemid
    active_items = ratings.loc[ratings.userid == userid].itemid.to_list()
    candidates = np.setdiff1d(Gu_items, active_items, assume_unique=True)[:candidate_number]
        
    return candidates

### 定义打分函数

In [16]:
# 由于分数预测需要获得用户评分的均值，我们在这里提前准备好
mean = ratings.groupby(by='userid', as_index=False)['rating'].mean()
mean = mean.to_numpy()[:, 1]

In [17]:
np_ratings = ratings.to_numpy()
np_ratings

array([[ 0.00000000e+00,  0.00000000e+00,  5.00000000e+00,
         3.61029412e+00,  1.38970588e+00],
       [ 0.00000000e+00,  1.00000000e+00,  3.00000000e+00,
         3.61029412e+00, -6.10294118e-01],
       [ 0.00000000e+00,  2.00000000e+00,  4.00000000e+00,
         3.61029412e+00,  3.89705882e-01],
       ...,
       [ 9.42000000e+02,  1.18700000e+03,  3.00000000e+00,
         3.41071429e+00, -4.10714286e-01],
       [ 9.42000000e+02,  1.22700000e+03,  3.00000000e+00,
         3.41071429e+00, -4.10714286e-01],
       [ 9.42000000e+02,  1.32900000e+03,  3.00000000e+00,
         3.41071429e+00, -4.10714286e-01]])

In [18]:
def predict(userid, itemid):
    """
    predict what score userid would have given to itemid.
    
    :param
        - userid : user id for which we want to make prediction
        - itemid : item id on which we want to make prediction
        
    :return
        - r_hat : predicted rating of user userid on item itemid
    """
    user_similarities = similarities[userid] 
    user_neighbors = neighbors[userid]
    # get mean rating of user userid
    user_mean = mean[userid]
    
    # find users who rated item 'itemid'
    iratings = np_ratings[np_ratings[:, 1].astype('int') == itemid]
    
    # find similar users to 'userid' who rated item 'itemid'
    suri = iratings[np.isin(iratings[:, 0], user_neighbors)]
    
    # similar users who rated current item (surci)
    normalized_ratings = suri[:,4]
    indexes = [np.where(user_neighbors == uid)[0][0] for uid in suri[:, 0].astype('int')]
    sims = user_similarities[indexes]
    
    num = np.dot(normalized_ratings, sims)
    den = np.sum(np.abs(sims))
    
    if num == 0 or den == 0:
        return user_mean
    
    r_hat = user_mean + np.dot(normalized_ratings, sims) / np.sum(np.abs(sims))
    
    return r_hat

### 计算每个用户对候选推荐商品的预测评分

In [19]:
def add_single_user_predictions(userid, predictions):
    """
    Make rating prediction for the active user on each candidate item and save in file prediction.csv
    
    :param
        - userid : id of the active user
        - predictions : where to save predictions
    """    
    # find candidate items for the active user
    candidates = find_candidate_items(userid)
    
    # loop over candidates items to make predictions
    for itemid in candidates:
        
        # prediction for userid on itemid
        r_hat = predict(userid, itemid)
        
        # save predictions
        predictions.append((userid, itemid, r_hat))
    return predictions


In [20]:
import sys

def user_predictions():
    """
    Make predictions for each user in the database.    
    """
    # get list of users in the database
    users = ratings.userid.unique()
    
    def _progress(count):
        sys.stdout.write('\rRating predictions. Progress status : %.1f%%' % (float(count/len(users))*100.0))
        sys.stdout.flush()
        
    predictions = []
    
    for count, userid in enumerate(users):        
        # make rating predictions for the current user
        predictions = add_single_user_predictions(userid, predictions)
        _progress(count)
        
    predictions = pd.DataFrame(predictions, columns=['userid', 'itemid', 'predicted_rating'])
    return predictions

In [21]:
predictions = user_predictions()

Rating predictions. Progress status : 99.9%

In [22]:
predictions

,userid,itemid,predicted_rating
0,0,404,3.545405
1,0,567,3.813794
2,0,422,3.936685
3,0,384,3.330252
4,0,287,3.774287
...,...,...,...
28285,942,237,4.269919
28286,942,450,2.727457
28287,942,287,3.826250
28288,942,163,3.417272


### 取最高的推荐


In [23]:
def top_ucf_recommendation(userid, predictions):
    """
    make top N recommendation for the active user.
    
    :param
        - userid : user id (original)
        - predictions : topN recommendations (original item id) reordered according to rating predictions
        
    :return
        - r_hat : predicted rating of user userid on item itemid
    """
    
    uid = uencoder.transform([userid])[0]
    
    predictions_user = predictions[predictions.userid==uid]
    List = predictions_user.sort_values(by=['predicted_rating'], ascending=False)
    
    List.userid = uencoder.inverse_transform(List.userid.tolist())
    List.itemid = iencoder.inverse_transform(List.itemid.tolist())
    
    List = pd.merge(List, movies, on='itemid', how='inner')
    
    return List

In [24]:
top_ucf_recommendation(212, predictions)

,userid,itemid,predicted_rating,title
0,212,173,4.754600,"Princess Bride, The (1987)"
1,212,97,4.734844,Dances with Wolves (1990)
2,212,174,4.710558,Raiders of the Lost Ark (1981)
3,212,71,4.699357,"Lion King, The (1994)"
4,212,196,4.678519,Dead Poets Society (1989)
5,212,95,4.478076,Aladdin (1992)
6,212,50,4.468001,Star Wars (1977)
7,212,194,4.407704,"Sting, The (1973)"
8,212,172,4.382282,"Empire Strikes Back, The (1980)"
9,212,181,4.358544,Return of the Jedi (1983)


# <b><span style='color:#F1C40F'>|</span> 基于近邻算法的协同过滤 - ICF</b>
本节内容修改于:

 📌 [Item-basedCollaborativeFiltering](https://github.com/nzhinusoftcm/review-on-collaborative-filtering/blob/master/3.Item-basedCollaborativeFiltering.ipynb)

### 得到评分矩阵

注意在这里我们要计算商品之间的相似度，所以第一个维度应该是item

In [25]:
def get_csr_rating_matrix(ratings):   
    return csr_matrix(pd.crosstab(ratings.itemid, ratings.userid, ratings.rating, aggfunc=sum).fillna(0).values)    

rating_matrix = get_csr_rating_matrix(ratings)


In [26]:
metric = 'cosine'
model = create_model(rating_matrix, k=21, metric=metric)
similarities, neighbors = nearest_neighbors(rating_matrix, model)
print('neighbors shape : ', neighbors.shape)
print('similarities shape : ', similarities.shape)

neighbors shape :  (1682, 21)
similarities shape :  (1682, 21)


### 定义找到推荐候选商品的函数

在这里我们定义同时符合以下条件的商品为候选推荐商品：

- 不属于目标用户已经购买过的商品 $I_u$
- 属于所有 $I_u$ 里的商品的相似商品的并集
- 属于top相似于 $I_u$ 的商品

In [27]:
def all_candidate_items(userid):
    """
    :param userid : user id for which we wish to find candidate items    
    :return : I_u, candidates
    """
    
    # 1. Finding the set I_u of items already rated by user userid
    I_u = np_ratings[np_ratings[:, 0] == userid]
    I_u = I_u[:, 1].astype('int')
    
    # 2. Taking the union of similar items for all items in I_u to form the set of candidate items
    c = set()
        
    for iid in I_u:    
        # add the neighbors of item iid in the set of candidate items
        c.update(neighbors[iid])
        
    c = list(c)
    # 3. exclude from the set C all items in I_u.
    candidates = np.setdiff1d(c, I_u, assume_unique=True)
    
    return I_u, candidates

test_user = uencoder.transform([1])[0]
i_u, u_candidates = all_candidate_items(test_user)
print('number of items purchased by user 1 : ', len(i_u))
print('number of all candidate items for user 1 : ', len(u_candidates))

number of items purchased by user 1 :  272
number of all candidate items for user 1 :  654


In [28]:
def similarity_with_Iu(c, I_u):
    """
    compute similarity between an item c and a set of items I_u. For each item i in I_u, get similarity between 
    i and c, if c exists in the set of items similar to itemid.    
    :param c : itemid of a candidate item
    :param I_u : set of items already purchased by a given user    
    :return w : similarity between c and I_u
    """
    w = 0    
    for iid in I_u :        
        # get similarity between itemid and c, if c is one of the k nearest neighbors of itemid
        if c in neighbors[iid] :
            w = w + similarities[iid, neighbors[iid] == c][0]    
    return w

def rank_candidates_by_similarity_with_Iu(candidates, I_u):
    """
    rank candidate items according to their similarities with i_u    
    :param candidates : list of candidate items
    :param I_u : list of items purchased by the user    
    :return ranked_candidates : dataframe of candidate items, ranked in descending order of similarities with I_u
    """
    
    # list of candidate items mapped to their corresponding similarities to I_u
    sims = [similarity_with_Iu(c, I_u) for c in candidates]
    candidates = iencoder.inverse_transform(candidates)    
    mapping = list(zip(candidates, sims))
    
    ranked_candidates = sorted(mapping, key=lambda couple:couple[1], reverse=True)    
    return ranked_candidates

def find_candidate_items(userid, candidate_number=30):
    """
    Produce top-N recommendation for a given user    
    :param userid : user for which we produce top-N recommendation
    :param n : length of the top-N recommendation list    
    :return topn
    """
    # find candidate items
    I_u, candidates = all_candidate_items(userid)
    
    # rank candidate items according to their similarities with I_u
    ranked_candidates = rank_candidates_by_similarity_with_Iu(candidates, I_u)
    
    # get the first N row of ranked_candidates to build the top N recommendation list
    topn = pd.DataFrame(ranked_candidates[:candidate_number], columns=['itemid','similarity_with_Iu'])    
    topn = pd.merge(topn, movies, on='itemid', how='inner')    
    return topn

In [29]:
def predict(userid, itemid):
    """
    Make rating prediction for user userid on item itemid    
    :param userid : id of the active user
    :param itemid : id of the item for which we are making prediction        
    :return r_hat : predicted rating
    """
    
    # Get items similar to item itemid with their corresponding similarities
    item_neighbors = neighbors[itemid]
    item_similarities = similarities[itemid]
    
    # get ratings of user with id userid
    uratings = np_ratings[np_ratings[:, 0].astype('int') == userid]
    
    # similar items rated by item the user of i
    siru = uratings[np.isin(uratings[:, 1], item_neighbors)]
    scores = siru[:, 2]
    indexes = [np.where(item_neighbors == iid)[0][0] for iid in siru[:,1].astype('int')]    
    sims = item_similarities[indexes]
    
    dot = np.dot(scores, sims)
    som = np.sum(np.abs(sims))

    if dot == 0 or som == 0:
        return mean[userid]
    
    return dot / som

In [30]:
def top_icf_recommendation(userid):
    """
    :param userid : id of the active user    
    :return topn : initial topN recommendations returned by the function item2item_topN
    :return topn_predict : topN recommendations reordered according to rating predictions
    """
    # make top N recommendation for the active user
    item_candidates = find_candidate_items(userid)
    
    # get list of items of the top N list
    itemids = item_candidates.itemid.to_list()
    
    predictions = []
    
    # make prediction for each item in the top N list
    for itemid in itemids:
        r = predict(userid, itemid)
        
        predictions.append((itemid,r))
    
    predictions = pd.DataFrame(predictions, columns=['itemid','prediction'])
    
    # merge the predictions to topN_list and rearrange the list according to predictions
    candidate_predictions = pd.merge(item_candidates, predictions, on='itemid', how='inner')
    candidate_predictions = candidate_predictions.sort_values(by=['prediction'], ascending=False)
    
    return candidate_predictions

In [31]:
test_user = uencoder.transform([1])[0]
top_icf_recommendation(userid=test_user)

,itemid,similarity_with_Iu,title,prediction
10,546,10.207321,Broken Arrow (1996),5.000000
26,288,6.565468,Scream (1996),5.000000
23,318,6.863154,Schindler's List (1993),5.000000
6,474,11.059820,Dr. Strangelove or: How I Learned to Stop Worr...,4.735340
29,514,6.508386,Annie Hall (1977),4.603404
25,300,6.648774,Air Force One (1997),4.467646
27,275,6.564902,Sense and Sensibility (1995),4.441006
11,276,10.182872,Leaving Las Vegas (1995),4.417165
13,285,9.663934,Secrets & Lies (1996),4.348225
17,286,8.987699,"English Patient, The (1996)",4.312252
